In [2]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

2023-07-23 23:10:00.281796: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 23:10:00.689482: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-23 23:10:00.691248: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-23 23:10:02.266286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

283 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [4]:

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [5]:

# shuffle our features and turn into np.array
random.shuffle(training)

patterns = [item[0] for item in training]
intents = [item[1] for item in training]
# create train and test lists. X - patterns, Y - intents
train_x = np.array(patterns)
train_y = np.array(intents)
print("Training data created")


Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


2023-07-23 23:11:32.074830: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [7]:
# Compile model. Use the updated SGD optimizer with the learning_rate argument.
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [8]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/200
57/57 [==============================] - 1s 2ms/step - loss: 4.8566 - accuracy: 0.0106
Epoch 2/200
57/57 [==============================] - 0s 3ms/step - loss: 4.7902 - accuracy: 0.0495
Epoch 3/200
57/57 [==============================] - 0s 2ms/step - loss: 4.6932 - accuracy: 0.0495
Epoch 4/200
57/57 [==============================] - 0s 2ms/step - loss: 4.6103 - accuracy: 0.0495
Epoch 5/200
57/57 [==============================] - 0s 2ms/step - loss: 4.4834 - accuracy: 0.0777
Epoch 6/200
57/57 [==============================] - 0s 2ms/step - loss: 4.3725 - accuracy: 0.0919
Epoch 7/200
57/57 [==============================] - 0s 2ms/step - loss: 4.2232 - accuracy: 0.0883
Epoch 8/200
57/57 [==============================] - 0s 2ms/step - loss: 4.0748 - accuracy: 0.1166
Epoch 9/200
57/57 [==============================] - 0s 2ms/step - loss: 3.9177 - accuracy: 0.1095
Epoch 10/200
57/57 [==============================] - 0s 2ms/step - loss: 3.8508 - accuracy: 0.1307
Epoch 11/

57/57 [==============================] - 0s 2ms/step - loss: 0.9494 - accuracy: 0.6996
Epoch 84/200
57/57 [==============================] - 0s 2ms/step - loss: 1.0078 - accuracy: 0.7138
Epoch 85/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8570 - accuracy: 0.7703
Epoch 86/200
57/57 [==============================] - 0s 2ms/step - loss: 0.9413 - accuracy: 0.7173
Epoch 87/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8534 - accuracy: 0.7244
Epoch 88/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8466 - accuracy: 0.7562
Epoch 89/200
57/57 [==============================] - 0s 2ms/step - loss: 0.9939 - accuracy: 0.6890
Epoch 90/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7320 - accuracy: 0.7880
Epoch 91/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8434 - accuracy: 0.7244
Epoch 92/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7403 - accuracy: 0.7633
Epoch 93/200


57/57 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.7880
Epoch 165/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7253 - accuracy: 0.7809
Epoch 166/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8784 - accuracy: 0.7244
Epoch 167/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8196 - accuracy: 0.7633
Epoch 168/200
57/57 [==============================] - 0s 2ms/step - loss: 0.6431 - accuracy: 0.8127
Epoch 169/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7312 - accuracy: 0.7597
Epoch 170/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7610 - accuracy: 0.7597
Epoch 171/200
57/57 [==============================] - 0s 2ms/step - loss: 0.8161 - accuracy: 0.7880
Epoch 172/200
57/57 [==============================] - 0s 2ms/step - loss: 0.7393 - accuracy: 0.7915
Epoch 173/200
57/57 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.7986
Epoc

/home/dimuli/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
